In [54]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import keras
from keras import layers
import tensorflow as tf

In [60]:
tf.config.list_physical_devices('GPU')

[]

In [43]:
df = pd.read_csv("../data/cleaned_extrovert.csv", engine='pyarrow')
text = df['post']
labels = df['label']


,post,label
0,"question, doctor, how'd get painkillers? otc p...",0
1,butt covid + cycle. i'm sure what's going i've...,0
2,different doctors. situation sucks relate peop...,0
3,thought pebbleyeet guy autistic guy wants “fix...,0
4,…i always end voting wrong even crewmate. hour...,0
...,...,...
40447,professional association** allow personal proj...,0
40448,work artist's behalf. please post [weekly musi...,0
40449,* chill * alternative/alt * lofi * electronic ...,0
40450,without ability appeal. ___ ___ questions beli...,0


In [35]:
X_train, X_test, y_train, y_test = train_test_split(text, labels, test_size=0.2, random_state=5)

In [36]:
max_features = 20000
embedding_dim = 128
sequence_length = 500

In [37]:
vectorize_layer = keras.layers.TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length
)

In [38]:
vectorize_layer.adapt(X_train)

In [39]:
# text_input = keras.Input(shape=(1,), dtype=tf.string, name='text')
# x = vectorize_layer(text_input)
# x = layers.Embedding(max_features + 1, embedding_dim)(x)

In [56]:
text_input = keras.Input(shape=(1,), dtype=tf.string, name='text')
x = vectorize_layer(text_input)
x = layers.Embedding(max_features + 1, embedding_dim)(x)

x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = keras.Model(text_input, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy", keras.metrics.AUC()])

In [57]:
epochs = 2
model.fit(X_train, y_train, epochs=epochs)

Epoch 1/2
1012/1012 ━━━━━━━━━━━━━━━━━━━━ 120s 117ms/step - accuracy: 0.7776 - auc: 0.6104 - loss: 0.5156
Epoch 2/2
1012/1012 ━━━━━━━━━━━━━━━━━━━━ 111s 110ms/step - accuracy: 0.8500 - auc: 0.8722 - loss: 0.3527


In [51]:
y_pred = model.predict(X_test)

253/253 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step


In [55]:
from sklearn.metrics import confusion_matrix
predictions = np.where(y_pred > 0.5, 1, 0)
confusion_matrix(y_test, predictions)

array([[6036,  216],
       [1146,  693]], dtype=int64)